# Latent Dirichlet allocation (LDA)
Latent Dirichlet allocation (LDA) is a topic model which infers topics from a collection of text documents. LDA can be thought of as a clustering algorithm as follows:  
*  Topics correspond to cluster centers, and documents correspond to examples (rows) in a dataset.
*  Topics and documents both exist in a feature space, where feature vectors are vectors of word counts (bag of words).
*  Rather than estimating a clustering using a traditional distance, LDA uses a function based on a statistical model of how text documents are generated.  

LDA supports different inference algorithms via setOptimizer function. EMLDAOptimizer learns clustering using expectation-maximization on the likelihood function and yields comprehensive results, while OnlineLDAOptimizer uses iterative mini-batch sampling for online variational inference and is generally memory friendly.  

LDA takes in a collection of documents as vectors of word counts and the following parameters (set using the builder pattern):

* k: Number of topics (i.e., cluster centers)
* optimizer: Optimizer to use for learning the LDA model, either EMLDAOptimizer or OnlineLDAOptimizer
* docConcentration: Dirichlet parameter for prior over documents’ distributions over topics. Larger values encourage smoother inferred distributions.
* topicConcentration: Dirichlet parameter for prior over topics’ distributions over terms (words). Larger values encourage smoother inferred distributions.
* maxIterations: Limit on the number of iterations.
* checkpointInterval: If using checkpointing (set in the Spark configuration), this parameter specifies the frequency with which checkpoints will be created. If maxIterations is large, using checkpointing can help reduce shuffle file sizes on disk and help with failure recovery.

All of MLlib’s LDA models support:

* describeTopics: Returns topics as arrays of most important terms and term weights
* topicsMatrix: Returns a vocabSize by k matrix where each column is a topic

Note: LDA is still an experimental feature under active development. As a result, certain features are only available in one of the two optimizers / models generated by the optimizer. Currently, a distributed model can be converted into a local model, but not vice-versa.  

The following discussion will describe each optimizer/model pair separately.  

### Expectation Maximization  
Implemented in EMLDAOptimizer and DistributedLDAModel.  
For the parameters provided to LDA:  
* docConcentration: Only symmetric priors are supported, so all values in the provided k-dimensional vector must be identical. All values must also be >1.0>1.0. Providing Vector(-1) results in default behavior (uniform k dimensional vector with value (50/k)+1(50/k)+1
* topicConcentration: Only symmetric priors supported. Values must be >1.0>1.0. Providing -1 results in defaulting to a value of 0.1+10.1+1.
* maxIterations: The maximum number of EM iterations.  

EMLDAOptimizer produces a DistributedLDAModel, which stores not only the inferred topics but also the full training corpus and topic distributions for each document in the training corpus. A DistributedLDAModel supports:  

* topTopicsPerDocument: The top topics and their weights for each document in the training corpus
* topDocumentsPerTopic: The top documents for each topic and the corresponding weight of the topic in the documents.
* logPrior: log probability of the estimated topics and document-topic distributions given the hyperparameters docConcentration and topicConcentration
* logLikelihood: log likelihood of the training corpus, given the inferred topics and document-topic distributions  

### Online Variational Bayes
Implemented in OnlineLDAOptimizer and LocalLDAModel.  
For the parameters provided to LDA:  
* docConcentration: Asymmetric priors can be used by passing in a vector with values equal to the Dirichlet parameter in each of the k dimensions. Values should be >=0>=0. Providing Vector(-1) results in default behavior (uniform k dimensional vector with value (1.0/k)(1.0/k))
* topicConcentration: Only symmetric priors supported. Values must be >=0>=0. Providing -1 results in defaulting to a value of (1.0/k)(1.0/k).
* maxIterations: Maximum number of minibatches to submit.  

In addition, OnlineLDAOptimizer accepts the following parameters:  

* miniBatchFraction: Fraction of corpus sampled and used at each iteration
* optimizeDocConcentration: If set to true, performs maximum-likelihood estimation of the hyperparameter docConcentration (aka alpha) after each minibatch and sets the optimized docConcentration in the returned LocalLDAModel
* tau0 and kappa: Used for learning-rate decay, which is computed by $(\tau_0 + iter)^{-\kappa}$ where iteriter is the current number of iterations.  

OnlineLDAOptimizer produces a LocalLDAModel, which only stores the inferred topics. A LocalLDAModel supports:  

* logLikelihood(documents): Calculates a lower bound on the provided documents given the inferred topics.
* logPerplexity(documents): Calculates an upper bound on the perplexity of the provided documents given the inferred topics.  

### Examples
In the following example, we load word count vectors representing a corpus of documents. We then use LDA to infer three topics from the documents. The number of desired clusters is passed to the algorithm. We then output the topics, represented as probability distributions over words.



In [1]:
val PATH = "file:///Users/lzz/work/SparkML/"

import org.apache.spark.mllib.clustering.{LDA, DistributedLDAModel}
import org.apache.spark.mllib.linalg.Vectors

// Load and parse the data
val data = sc.textFile( PATH + "data/mllib/sample_lda_data.txt")
val parsedData = data.map(s => Vectors.dense(s.trim.split(' ').map(_.toDouble)))
// Index documents with unique IDs
val corpus = parsedData.zipWithIndex.map(_.swap).cache()

// Cluster the documents into three topics using LDA
val ldaModel = new LDA().setK(3).run(corpus)

// Output topics. Each is a distribution over words (matching word count vectors)
println("Learned topics (as distributions over vocab of " + ldaModel.vocabSize + " words):")
val topics = ldaModel.topicsMatrix
for (topic <- Range(0, 3)) {
  print("Topic " + topic + ":")
  for (word <- Range(0, ldaModel.vocabSize)) { print(" " + topics(word, topic)); }
  println()
}

// Save and load model.
ldaModel.save(sc, "myLDAModel")
val sameModel = DistributedLDAModel.load(sc, "myLDAModel")

Learned topics (as distributions over vocab of 11 words):
Topic 0: 11.082131945394131 13.096273230955001 1.7828503882078757 5.335951336627549 13.554427641829196 5.244268769875691 6.647561701256702 3.432112549375341 2.9725230849045166 8.782699890300785 14.155634058335988
Topic 1: 7.649720574244201 4.270419292006031 2.6589606318530206 32.830920089898314 4.0704441118472054 5.322411996044803 8.22113549731758 1.5225070821672357 2.241963856600081 6.9539310872312585 10.94090835710875
Topic 2: 7.268147480361669 11.633307477038969 7.5581889799391035 1.8331285734741423 7.375128246323598 11.433319234079507 16.131302801425722 5.0453803684574225 2.785513058495403 8.263369022467955 7.903457584555262
